In [3]:
import paramiko
import pandas as pd
from pathlib import Path
import paramiko
from paramiko import SSHException
import json
def connect_sftp(host, port, username, password):
    try:
        # 建立 SSH 客戶端
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        
        # 連接到伺服器
        ssh.connect(hostname=host, port=port, username=username, password=password)
        
        # 開啟 SFTP session
        sftp = ssh.open_sftp()
        
        print("SFTP 連線成功")
        return sftp
    except Exception as e:
        print(f"連接 SFTP 時發生錯誤: {e}")
        return None

SFTP_HOST = '60.250.194.37'    # 替換為您的 SFTP 伺服器地址
SFTP_PORT = 5022                         # 一般 SFTP 使用 22 埠
SFTP_USERNAME = 'itri'        # 替換為您的使用者名稱
SFTP_PASSWORD = 'nctu503'        # 替換為您的密碼


In [5]:
# 設定遠端基礎目錄
REMOTE_BASE_DIR = '/itri/Taiwan Water Local/Hsinchu City'
LOCAL_BASE_DIR = Path('dataset/ftp')

# 讀取 CSV 檔案
CSV_FILE_PATH = 'map_location_web_ai_result70.csv'  # 替換為您的 CSV 檔案路徑

def get_ai_result_dir(ai_result):
    """
    根據 ai_result 計算 ai_result_dir。
    ai_result 在 50~59 為 50，60~69 為 60，以此類推。
    """
    try:
        ai_result_int = int(float(ai_result))
        ai_result_dir = (ai_result_int // 10) * 10
        return str(ai_result_dir)
    except ValueError:
        return 'unknown'

In [19]:
def download_wav_files(csv_path):
    # 讀取 CSV
    df = pd.read_csv(csv_path)

    # 初始化 SFTP 連線
    transport = paramiko.Transport((SFTP_HOST, SFTP_PORT))
    try:
        transport.connect(username=SFTP_USERNAME, password=SFTP_PASSWORD)
        sftp = paramiko.SFTPClient.from_transport(transport)
        print("SFTP 連線成功")

        for index, row in df.iterrows():
            try:
                serial_number = row['serial_number']
                date = row['date']  # 假設 date 格式為 'YYYY-MM-DD'
                wav_data = row['wav_data']
                ai_result = row['ai_result']

                # 計算 ai_result_dir
                ai_result_dir = get_ai_result_dir(ai_result)

                # 設定本地儲存路徑
                local_dir = LOCAL_BASE_DIR / ai_result_dir / serial_number
                local_dir.mkdir(parents=True, exist_ok=True)

                # 解析 wav_data（假設 wav_data 包含檔案名稱或路徑）
                # 根據您的 CSV 範例，wav_data 似乎是 JSON 格式的字串
                # 您需要根據實際情況調整解析方式

                wav_file_name = wav_data.strip()

                # 構建遠端檔案路徑
                remote_path = f"{REMOTE_BASE_DIR}/{serial_number}/WAV/{date}/{wav_file_name}.wav"

                # 設定本地檔案路徑
                local_file_path = local_dir / f"{wav_file_name}.wav"

                # 下載檔案
                print(f"下載第 {index + 1} 行：{remote_path} 到 {local_file_path}")
                sftp.get(remote_path, str(local_file_path))
                print(f"下載成功：{local_file_path}")

            except Exception as e:
                print(f"第 {index + 1} 行下載失敗：{e}")

    except SSHException as ssh_e:
        print(f"SFTP 連線失敗：{ssh_e}")
    finally:
        transport.close()
        print("SFTP 連線已關閉")

In [ ]:
download_wav_files(CSV_FILE_PATH)

SFTP 連線成功
下載第 1 行：/itri/Taiwan Water Local/Hsinchu City/ITRI0004003/WAV/2024-07-01/2024-07-01T094426.wav 到 dataset\ftp\70\ITRI0004003\2024-07-01T094426.wav
下載成功：dataset\ftp\70\ITRI0004003\2024-07-01T094426.wav
下載第 2 行：/itri/Taiwan Water Local/Hsinchu City/ITRI0004003/WAV/2024-07-01/2024-07-01T094502.wav 到 dataset\ftp\80\ITRI0004003\2024-07-01T094502.wav
下載成功：dataset\ftp\80\ITRI0004003\2024-07-01T094502.wav
下載第 3 行：/itri/Taiwan Water Local/Hsinchu City/ITRI0004003/WAV/2024-07-01/2024-07-01T094525.wav 到 dataset\ftp\80\ITRI0004003\2024-07-01T094525.wav
下載成功：dataset\ftp\80\ITRI0004003\2024-07-01T094525.wav
下載第 4 行：/itri/Taiwan Water Local/Hsinchu City/ITRI0004003/WAV/2024-07-01/2024-07-01T094530.wav 到 dataset\ftp\70\ITRI0004003\2024-07-01T094530.wav
下載成功：dataset\ftp\70\ITRI0004003\2024-07-01T094530.wav
下載第 5 行：/itri/Taiwan Water Local/Hsinchu City/ITRI0003003/WAV/2024-07-01/2024-07-01T101859.wav 到 dataset\ftp\70\ITRI0003003\2024-07-01T101859.wav
下載成功：dataset\ftp\70\ITRI0003003\2024-07-01T1